In [1]:
import pandas as pd
import requests
from datetime import datetime, timedelta
import time

In [2]:
def get_historical_data(unit, count, to=None):
    url = f"https://api.upbit.com/v1/candles/minutes/{unit}?market=KRW-BTC&to={to}&count={count}"
    headers = {"accept": "application/json"}

    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Error: {response.json()}")
    data = response.json()
    return data

def fetch_all_historical_data(unit, start_date, end_date):
    start = datetime.strptime(start_date, '%Y-%m-%dT %H:%M:%S')
    all_data = []
    to_date = start.strftime('%Y-%m-%dT %H:%M:%S')

    end = datetime.strptime(end_date, '%Y-%m-%dT %H:%M:%S')

    while datetime.strptime(to_date, '%Y-%m-%dT %H:%M:%S') > end:
        data = get_historical_data(unit=unit, count=200, to=to_date)
        all_data.extend(data)
        last_candle_time = datetime.strptime(data[-1]['candle_date_time_utc'], '%Y-%m-%dT %H:%M:%S')
        to_date = (last_candle_time - timedelta(minutes=unit)).strftime('%Y-%m-%dT %H:%M:%S')
        time.sleep(1)
        
    return all_data

# Example: Fetch all 1-minute interval data from 2024-06-15 to 2017-10-01
end_date = '2017-10-01T 00:00:00'
start_date = '2024-06-15T 23:59:59'
historical_data = fetch_all_historical_data(unit=60, start_date=start_date, end_date=end_date)


Error: {'error': {'name': 400, 'message': 'Invalid parameter. Check the given value!'}}


KeyError: -1

In [3]:
df=pd.DataFrame(historical_data)
df['candle_date_time_kst'] = pd.to_datetime(df['candle_date_time_kst'], format='%Y-%m-%dT%H:%M:%S')
df = df.sort_values(by='candle_date_time_kst')
# df = df.drop_duplicates().reset_index(drop=True)
df.reset_index(drop=True)

NameError: name 'historical_data' is not defined